The code from lab 7 is used in this assignement 

In [215]:
!pip3 install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [216]:
#Importing libarys from lab 7 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [0]:
BATCH_SIZE = 200

## transformations
transform = transforms.Compose(
    [transforms.ToTensor()])

## download and load training dataset
trainset = torchvision.datasets.FashionMNIST(root='./data/FshionMNIST', download=True, transform=transforms.Compose([transforms.ToTensor()]))
testset = torchvision.datasets.FashionMNIST(root='./data/FshionMNIST', download=True, train=False,  transform=transforms.Compose([transforms.ToTensor()]))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

batch = next(iter(trainloader))
images, labels = batch 

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE)
                                          

## download and load testing dataset
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)



In [0]:
class NnDigitsClassifier(nn.Module):
    def __init__(self):
        super(NnDigitsClassifier, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.d1 = nn.Linear(26 * 26 * 32, 128)
        self.d2 = nn.Linear(128, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = F.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = F.relu(x)

        # logits => 32x10
        logits = self.d2(x)
        out = F.softmax(logits, dim=1)
        return out

Training 

In [219]:
## test the model with 1 batch
model = NnDigitsClassifier()
for images, labels in trainloader:
    print("batch size:", images.shape)
    out = model(images)
    print(out.shape)
    break


batch size: torch.Size([200, 1, 28, 28])
torch.Size([200, 10])


In [0]:
learning_rate = 0.0019
num_epochs = 20

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = NnDigitsClassifier()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## compute accuracy
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [221]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i))

Epoch: 0 | Loss: 1.6815 | Train Accuracy: 79.06
Epoch: 1 | Loss: 1.5950 | Train Accuracy: 87.60
Epoch: 2 | Loss: 1.5760 | Train Accuracy: 89.48
Epoch: 3 | Loss: 1.5650 | Train Accuracy: 90.60
Epoch: 4 | Loss: 1.5553 | Train Accuracy: 91.57
Epoch: 5 | Loss: 1.5487 | Train Accuracy: 92.23
Epoch: 6 | Loss: 1.5418 | Train Accuracy: 92.91
Epoch: 7 | Loss: 1.5362 | Train Accuracy: 93.47
Epoch: 8 | Loss: 1.5327 | Train Accuracy: 93.83
Epoch: 9 | Loss: 1.5301 | Train Accuracy: 94.06
Epoch: 10 | Loss: 1.5244 | Train Accuracy: 94.64
Epoch: 11 | Loss: 1.5220 | Train Accuracy: 94.85
Epoch: 12 | Loss: 1.5192 | Train Accuracy: 95.13
Epoch: 13 | Loss: 1.5165 | Train Accuracy: 95.37
Epoch: 14 | Loss: 1.5150 | Train Accuracy: 95.55
Epoch: 15 | Loss: 1.5141 | Train Accuracy: 95.61
Epoch: 16 | Loss: 1.5114 | Train Accuracy: 95.90
Epoch: 17 | Loss: 1.5101 | Train Accuracy: 96.02
Epoch: 18 | Loss: 1.5094 | Train Accuracy: 96.05
Epoch: 19 | Loss: 1.5073 | Train Accuracy: 96.27


In [222]:
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        
print('Test Accuracy: %.2f'%( test_acc/i))

Test Accuracy: 92.00


Saving and loading the model
1. Source: https://pytorch.org/docs/stable/torch.html?highlight=torch%20load#torch.save

In [223]:
model = NnDigitsClassifier()
torch.save(model.state_dict(), 'saved_model_state.pt')
#torch.save(model.state_dict, 'fashionMNIST')

model.load_state_dict(torch.load('saved_model_state.pt'))

<All keys matched successfully>

The confusion matrix (not working properly) 
SOURCE: https://deeplizard.com/learn/video/0LhiS6yu2qQ


In [0]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [0]:
net = NnDigitsClassifier()

In [226]:
len(trainset)
len(trainset.targets)

60000

In [0]:
def get_all_preds (model, loader):
  all_preds = torch.tensor([])
  for batch in loader: 
    images, labels = batch

    preds = model(images)
    all_preds = torch.cat(
        (all_preds, preds)
        , dim=0
        )
  return all_preds

In [0]:
with torch.no_grad():
  prediction_loader = torch.utils.data.DataLoader(trainset, batch_size=10000)
  train_preds = get_all_preds(net, prediction_loader)

In [229]:
print(train_preds.requires_grad)

False


In [0]:
train_preds.grad

In [0]:
train_preds.grad_fn

In [232]:
preds_correct = get_num_correct(train_preds, trainset.targets)

print("total correct", preds_correct)
print("accuracy:", preds_correct/len(trainset))


total correct 5791
accuracy: 0.09651666666666667


BUILDING MATRIX

In [233]:
trainset.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [234]:
train_preds.argmax(dim=1)

tensor([1, 1, 7,  ..., 7, 7, 7])

In [0]:
stacked = torch.stack(
    (
        trainset.targets
        ,train_preds.argmax(dim=1)
    )
    ,dim=1
)

In [236]:
stacked.shape

torch.Size([60000, 2])

In [237]:
stacked

tensor([[9, 1],
        [0, 1],
        [0, 7],
        ...,
        [3, 7],
        [0, 7],
        [5, 7]])

In [238]:
stacked[0].tolist()

[9, 1]

In [239]:
cmt = torch.zeros(10,10, dtype=torch.int64)
cmt

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [0]:
for p in stacked: 
  tl, pl = p.tolist()
  cmt[tl, pl] = cmt[tl, pl] + 1

In [241]:
cmt

tensor([[   0, 1281,    0,  263,    0,    0,    0, 4456,    0,    0],
        [   0, 3521,    1, 1068,    0,    0,    0, 1410,    0,    0],
        [   0,  984,    0,  223,    0,    3,    0, 4790,    0,    0],
        [   0, 2398,    0,  718,    0,    0,    0, 2884,    0,    0],
        [   0, 1499,    0,  251,    0,    2,    0, 4248,    0,    0],
        [   0, 1691,    0,  509,    0,    4,    1, 3795,    0,    0],
        [   0,  780,    0,  333,    0,    0,    0, 4886,    0,    1],
        [   0, 4352,    0,  100,    0,    0,    0, 1548,    0,    0],
        [   0, 1580,    2, 1183,    0,    3,    0, 3232,    0,    0],
        [   0, 2695,    0,   33,    0,    0,    0, 3272,    0,    0]])

PLOTTING

In [0]:
#I tried to import the plotcm file from deeplizard.com, but i did not manage to import it in colab, but it should work
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from resources.plotcm import plot_confusion_matrix

In [0]:
cm = confusion_matrix(trainset.targets, train_preds.argmax(dim=1))
print(type(cm))
cm

<class 'numpy.ndarray'>


array([[   5, 5993,    0,    2,    0,    0,    0,    0,    0,    0],
       [   0, 5999,    0,    0,    0,    0,    0,    1,    0,    0],
       [   5, 5972,    0,   14,    0,    0,    0,    9,    0,    0],
       [   2, 5997,    0,    0,    0,    0,    0,    1,    0,    0],
       [   3, 5967,    0,   15,    0,    0,    0,   15,    0,    0],
       [  21, 5896,    0,   76,    0,    0,    0,    7,    0,    0],
       [   6, 5981,    0,    6,    0,    0,    0,    7,    0,    0],
       [   0, 5996,    0,    4,    0,    0,    0,    0,    0,    0],
       [   3, 5943,    0,   33,    0,    0,    0,   21,    0,    0],
       [   0, 5998,    0,    2,    0,    0,    0,    0,    0,    0]])

In [0]:
#names = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal'
#          'Sneaker', 'Bag', 'Ankle boot')
#plt.figure(figsize=(10,10))
#plot_confusion_matrix(cmt, names)

SOURCES 
1. https://pytorch.org/docs/master/torchvision/datasets.html#fashion-mnist
2. https://deeplizard.com/learn/video/0LhiS6yu2qQ
3. Lab 6 
4. Lab 7 
5. http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/
6. https://pytorch.org/docs/stable/nn.html 
7. https://github.com/zalandoresearch/fashion-mnist
8. https://pytorch.org/docs/stable/torch.html?highlight=torch%20load#torch.save